In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import BorderlineSMOTE
from xgboost import XGBClassifier
from sklearn.svm import SVC
import lightgbm
from sklearn.naive_bayes import GaussianNB,ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.tree import DecisionTreeClassifier

2025-03-15 10:27:25.415991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-15 10:27:25.426788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-15 10:27:25.430093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-15 10:27:25.439158: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
xtrain_df = pd.read_csv('task/train_set/x_train.csv')
xtrain_agg_df = pd.read_csv('task/train_set/x_train_aggregated.csv')
ytrain_df = pd.read_csv('task/train_set/y_train.csv')
xval_agg_df = pd.read_csv('task/validation_set/x_val_aggregated.csv')
yval_df = pd.read_csv('task/validation_set/y_val.csv')
xtrain_fea = pd.read_csv('aggregated_features.csv')


In [6]:
account_ids_df1 = xtrain_fea['AccountID']
account_ids_df2 = ytrain_df['AccountID']

# Check if all AccountID in df1 are in df2
all_matching = account_ids_df1.isin(account_ids_df2).all()
if all_matching:
    print("All AccountID in df1 are present in df2.")
else:
    print("Not all AccountID in df1 are present in df2.")

All AccountID in df1 are present in df2.


In [7]:
xy_train = pd.merge(xtrain_agg_df, ytrain_df, on='AccountID', how='inner')

In [4]:
xtrain_df.head()

,Hour,Action,Amount,AccountID,External,OldBalance,NewBalance,isUnauthorizedOverdraft
0,1,CASH_IN,133687.16,C9618460261,NaN,104703.85,238391.02,0
1,1,CASH_IN,171878.50,C0742280337,NaN,193747.95,365626.45,0
2,1,CASH_IN,110503.91,C1337418745,NaN,6.40,110510.31,0
3,1,CASH_IN,158292.33,C7477491662,NaN,87.03,158379.36,0
4,1,CASH_IN,208342.54,C7477491662,NaN,158379.36,366721.91,0


In [5]:
xtrain_df.describe()

,Hour,Amount,OldBalance,NewBalance,isUnauthorizedOverdraft
count,1.985971e+06,1.985971e+06,1.985971e+06,1.985971e+06,1.985971e+06
mean,1.850364e+02,1.379793e+05,3.047470e+06,3.037178e+06,3.496980e-02
std,1.256821e+02,3.026488e+05,2.049241e+06,2.014591e+06,1.837034e-01
min,1.000000e+00,2.000000e-02,-1.990282e+05,-1.990282e+05,0.000000e+00
25%,4.500000e+01,1.217673e+04,1.897720e+06,1.926878e+06,0.000000e+00
50%,1.870000e+02,9.046182e+04,3.219526e+06,3.211457e+06,0.000000e+00
75%,2.810000e+02,1.667487e+05,4.048426e+06,4.026252e+06,0.000000e+00
max,7.190000e+02,1.814506e+07,3.020602e+07,3.003953e+07,1.000000e+00


In [6]:
xtrain_agg_df.head()

,AccountID,NumTransactions,AvgAmount,MaxAmount,TotalIn,MaxIn,TotalOut,MaxCashOut,MaxDebit,MaxTransfer,MaxPayment,MaxFreqH,MaxFreqD
0,C0001143040,562,146658.532989,3585064.01,35124031.74,424498.32,47298063.80,284531.00,8893.26,3585064.01,27070.99,12,84
1,C0001332727,178,128888.997753,903276.53,13396604.25,341238.53,9545637.35,258056.33,0.00,903276.53,21396.14,6,33
2,C0002641069,56,146342.356786,2571512.49,4547603.60,293992.40,3647568.38,204095.40,3594.16,2571512.49,14710.21,2,8
3,C0002967890,103,117303.432330,3554689.54,3016367.57,392633.82,9065885.96,247445.60,23100.20,3554689.54,24640.16,4,22
4,C0003513711,50,144333.040800,1563406.66,4462829.33,264727.14,2753822.71,142728.98,3080.34,1563406.66,16098.56,2,8


In [7]:
xtrain_agg_df.describe()

,NumTransactions,AvgAmount,MaxAmount,TotalIn,MaxIn,TotalOut,MaxCashOut,MaxDebit,MaxTransfer,MaxPayment,MaxFreqH,MaxFreqD
count,11061.000000,1.106100e+04,1.106100e+04,1.106100e+04,1.106100e+04,1.106100e+04,1.106100e+04,11061.000000,1.106100e+04,11061.000000,11061.000000,11061.000000
mean,173.268421,1.454351e+05,1.866861e+06,1.042735e+07,3.393609e+05,1.227517e+07,2.308254e+05,5364.761463,1.815955e+06,19479.734623,5.223488,28.436037
std,1122.199246,6.548711e+04,1.318804e+06,2.292905e+07,5.875693e+05,2.354127e+07,1.153758e+05,7669.151693,1.290086e+06,7948.617402,17.342272,176.490148
min,1.000000,4.407557e+03,8.437110e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,1.000000,1.000000
25%,39.000000,1.196196e+05,8.641564e+05,2.128583e+06,2.599915e+05,2.558064e+06,1.981320e+05,0.000000,8.604498e+05,15773.650000,2.000000,7.000000
50%,99.000000,1.387191e+05,1.780288e+06,6.165977e+06,3.080791e+05,8.088935e+06,2.453406e+05,3058.480000,1.776725e+06,20428.490000,4.000000,17.000000
75%,215.000000,1.561108e+05,2.625486e+06,1.351976e+07,3.534323e+05,1.671086e+07,2.800967e+05,7951.330000,2.616498e+06,24225.720000,6.000000,35.000000
max,77542.000000,1.804926e+06,1.705500e+07,1.392348e+09,1.705500e+07,1.400163e+09,4.008213e+06,162019.560000,1.206550e+07,78382.870000,1206.000000,12265.000000


In [8]:
ytrain_df.describe()

,Fraudster
count,11061.000000
mean,0.157129
std,0.363938
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [9]:
xtrain_df.dtypes

Hour                         int64
Action                      object
Amount                     float64
AccountID                   object
External                    object
OldBalance                 float64
NewBalance                 float64
isUnauthorizedOverdraft      int64
dtype: object

In [10]:
xtrain_df.isnull().sum()

Hour                             0
Action                           0
Amount                           0
AccountID                        0
External                   1269776
OldBalance                       0
NewBalance                       0
isUnauthorizedOverdraft          0
dtype: int64

In [11]:
xtrain_agg_df.dtypes

AccountID           object
NumTransactions      int64
AvgAmount          float64
MaxAmount          float64
TotalIn            float64
MaxIn              float64
TotalOut           float64
MaxCashOut         float64
MaxDebit           float64
MaxTransfer        float64
MaxPayment         float64
MaxFreqH             int64
MaxFreqD             int64
dtype: object

In [12]:
fraudster_counts = ytrain_df['Fraudster'].value_counts()
fraudster_counts_sorted = fraudster_counts.sort_values(ascending=False)
print(fraudster_counts_sorted)


Fraudster
0    9323
1    1738
Name: count, dtype: int64


In [13]:
x = xtrain_agg_df[['NumTransactions','AvgAmount','MaxAmount','TotalIn','MaxIn','TotalOut','MaxCashOut','MaxDebit','MaxTransfer','MaxPayment','MaxFreqH','MaxFreqD']]
y = ytrain_df['Fraudster']
xval = xval_agg_df[['NumTransactions','AvgAmount','MaxAmount','TotalIn','MaxIn','TotalOut','MaxCashOut','MaxDebit','MaxTransfer','MaxPayment','MaxFreqH','MaxFreqD']]
yval = yval_df['Fraudster']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
scaler = StandardScaler().fit(x)
x = scaler.transform(x)
xval = scaler.transform(xval)

In [14]:
    #to handle class imbalance
smote = BorderlineSMOTE(random_state=42, kind='borderline-1')
x_train_smote, y_train_smote = smote.fit_resample(x, y)
x = x_train_smote
y = y_train_smote

In [15]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(x, y)
# y_pred = rf_model.predict(x_test)
# # Evaluation
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

RandomForestClassifier(random_state=42)

In [16]:

y_pred = rf_model.predict(xval)
# Evaluation
accuracy = accuracy_score(yval, y_pred)
print(f"Accuracy: {accuracy}")
print(confusion_matrix(yval, y_pred))
print(classification_report(yval, y_pred))

Accuracy: 0.8248643761301989
[[9064  344]
 [1593   59]]
              precision    recall  f1-score   support

           0       0.85      0.96      0.90      9408
           1       0.15      0.04      0.06      1652

    accuracy                           0.82     11060
   macro avg       0.50      0.50      0.48     11060
weighted avg       0.75      0.82      0.78     11060



In [17]:
# Feature Importance from rf model
feature_importance = pd.DataFrame({
    'Feature': x.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Feature Importance:")
print(feature_importance)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
# Plot Feature Importance
import seaborn as sns
import matplotlib.pyplot as plt
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title("Feature Importance from Random Forest")
plt.show()

NameError: name 'feature_importance' is not defined

In [18]:
X = np.array(x)
Y = np.array(y).reshape((11061,1))

print(X.shape)
print(Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

ValueError: cannot reshape array of size 18646 into shape (11061,1)

In [ ]:


X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # Reshaping to (samples, timesteps=1, features)
model.add(LSTM(64, input_shape=(X_train.shape[0], X_train.shape[1]), return_sequences=True))

model = Sequential()
model.add(LSTM(64,input_shape=(X_train.shape[0], X_train.shape[1]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(4, activation='softmax'))

model.summary()

2025-03-02 11:09:07.587538: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-02 11:09:07.666836: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-02 11:09:07.716900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740910147.774141   25477 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740910147.792272   25477 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 11:09:07.929303: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

NameError: name 'model' is not defined

In [ ]:
adam_optimizer = keras.optimizers.AdamW(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.1)

Epoch 1/100


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 12 and 1 for '{{node sequential_1/lstm_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1/lstm_1/strided_slice_2, sequential_1/lstm_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [?,12], [1,256].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(None, 12), dtype=float32)
  • states=('tf.Tensor(shape=(None, 64), dtype=float32)', 'tf.Tensor(shape=(None, 64), dtype=float32)')
  • training=True

In [ ]:
Y_pred = model.predict(X_test)

Y_pred = np.argmax(Y_pred, axis=1)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed evaluation metrics
print(classification_report(Y_test, Y_pred))

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 12), dtype=float32). Expected shape (None, 9954, 12), but input has incompatible shape (32, 12)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 12), dtype=float32)
  • training=False
  • mask=None

In [20]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Initialize the XGBoost classifier
# model_XG = SVC(verbose = True, C=100, decision_function_shape='ovo',class_weight='balanced',random_state=42)
model_XG = XGBClassifier(scale_pos_weight=10, learning_rate=0.05, n_estimators=200, max_depth=6, random_state=42)

# Train the model
model_XG.fit(x, y)

# Make predictions
y_XG_pred = model_XG.predict(xval)

# Evaluate the model
accuracy = accuracy_score(yval, y_XG_pred)
print(f"Accuracy: {accuracy}")
print(confusion_matrix(yval, y_pred))
print(classification_report(yval, y_pred))

Accuracy: 0.26112115732368896
[[9064  344]
 [1593   59]]
              precision    recall  f1-score   support

           0       0.85      0.96      0.90      9408
           1       0.15      0.04      0.06      1652

    accuracy                           0.82     11060
   macro avg       0.50      0.50      0.48     11060
weighted avg       0.75      0.82      0.78     11060



In [21]:

# Initialize the Naive Bayes classifier
nb_model = GaussianNB()

# clf_1 = SVC(kernel='linear', verbose = True, C=100, decision_function_shape='ovo', class_weight='balanced', random_state=42)
clf_1 = GaussianNB()
clf_2 = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='manhattan')
clf_3 = MLPClassifier(hidden_layer_sizes=(300, ), random_state=42)
clf_5 = LogisticRegression(random_state=42)
clf_4 = DecisionTreeClassifier(random_state=42)
# clf_2 = XGBClassifier(booster='gbtree', objective='multi:softmax', num_class=len(np.unique(y)), max_depth=6)
# clf_3 = lightgbm.LGBMClassifier(boosting_type='gbdt', objective='multiclass', num_class=len(np.unique(y)), random_state=42, num_leaves=31, force_col_wise=True)
# clf_4 = RandomForestClassifier(n_estimators=300, random_state=42, class_weight='balanced', criterion='gini')
eclf = VotingClassifier(estimators=[('svc', clf_1), ('xgb', clf_2), ('lgbm', clf_3), ('rf', clf_4),('rf2', clf_5)], voting='hard')


In [22]:
eclf.fit(x, y)
train_predictions = eclf.predict(x)
train_accuracy = accuracy_score(y_true=y, y_pred=train_predictions)
print("Training accuracy: ", train_accuracy)
test_predictions = eclf.predict(xval)
test_accuracy = accuracy_score(y_true=yval, y_pred=test_predictions)
print("Test accuracy: ", test_accuracy)
conf_matrix = confusion_matrix(y_true=yval, y_pred=test_predictions)
print("Confusion matrix: ", conf_matrix)
clf_report = classification_report(y_true=yval, y_pred=test_predictions)
print("Classification report: ", clf_report)

/home/harsha/anaconda3/envs/bbdc25/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training accuracy:  0.8982087310951411
Test accuracy:  0.5476491862567812
Confusion matrix:  [[5359 4049]
 [ 954  698]]
Classification report:                precision    recall  f1-score   support

           0       0.85      0.57      0.68      9408
           1       0.15      0.42      0.22      1652

    accuracy                           0.55     11060
   macro avg       0.50      0.50      0.45     11060
weighted avg       0.74      0.55      0.61     11060



In [23]:
# Initialize the Logistic Regression classifier
log_reg = LogisticRegression(random_state=42)

# Train the model
log_reg.fit(x, y)

# Make predictions
y_pred = log_reg.predict(xval)

# Evaluate the model
accuracy = accuracy_score(yval, y_pred)
print(f"Accuracy: {accuracy}")
print(confusion_matrix(yval, y_pred))
print(classification_report(yval, y_pred))

Accuracy: 0.4560578661844485
[[4123 5285]
 [ 731  921]]
              precision    recall  f1-score   support

           0       0.85      0.44      0.58      9408
           1       0.15      0.56      0.23      1652

    accuracy                           0.46     11060
   macro avg       0.50      0.50      0.41     11060
weighted avg       0.74      0.46      0.53     11060



In [24]:
# Initialize the KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='manhattan')

# Train the model
knn_model.fit(x, y)

# Make predictions
y_pred = knn_model.predict(xval)

# Evaluate the model
accuracy = accuracy_score(yval, y_pred)
print(f"Accuracy: {accuracy}")
print(confusion_matrix(yval, y_pred))
print(classification_report(yval, y_pred))

Accuracy: 0.6166365280289331
[[6286 3122]
 [1118  534]]
              precision    recall  f1-score   support

           0       0.85      0.67      0.75      9408
           1       0.15      0.32      0.20      1652

    accuracy                           0.62     11060
   macro avg       0.50      0.50      0.47     11060
weighted avg       0.74      0.62      0.67     11060



In [25]:
# Initialize the MLP classifier
mlp_model = MLPClassifier(hidden_layer_sizes=(200,), 
                      solver='adam', 
                      learning_rate='adaptive', 
                      max_iter=500, 
                      alpha=0.0001,
                      random_state=42)

# Train the model
mlp_model.fit(x, y)

# Make predictions
y_pred = mlp_model.predict(xval)

# Evaluate the model
accuracy = accuracy_score(yval, y_pred)
print(f"Accuracy: {accuracy}")
print(confusion_matrix(yval, y_pred))
print(classification_report(yval, y_pred))

Accuracy: 0.6015370705244123
[[6073 3335]
 [1072  580]]
              precision    recall  f1-score   support

           0       0.85      0.65      0.73      9408
           1       0.15      0.35      0.21      1652

    accuracy                           0.60     11060
   macro avg       0.50      0.50      0.47     11060
weighted avg       0.75      0.60      0.66     11060



/home/harsha/anaconda3/envs/bbdc25/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [26]:
# Initialize the Naive Bayes classifier
# nb_model = GaussianNB(var_smoothing=1e-9)

# nb_model = ComplementNB()


# Train the model
nb_model.fit(x, y)

# Make predictions
y_pred = nb_model.predict(xval)

# Evaluate the model
accuracy = accuracy_score(yval, y_pred)
print(f"Accuracy: {accuracy}")
print(confusion_matrix(yval, y_pred))
print(classification_report(yval, y_pred))

Accuracy: 0.20678119349005425
[[ 789 8619]
 [ 154 1498]]
              precision    recall  f1-score   support

           0       0.84      0.08      0.15      9408
           1       0.15      0.91      0.25      1652

    accuracy                           0.21     11060
   macro avg       0.49      0.50      0.20     11060
weighted avg       0.73      0.21      0.17     11060



In [27]:

# Initialize the Decision Tree classifier
dt_model = DecisionTreeClassifier(random_state=42)

# Train the model
dt_model.fit(x,y)

# Make predictions
y_pred = dt_model.predict(xval)

# Evaluate the model
accuracy = accuracy_score(yval, y_pred)
print(f"Accuracy: {accuracy}")
print(confusion_matrix(yval, y_pred))


Accuracy: 0.7039783001808319
[[7441 1967]
 [1307  345]]
